# Simple Twitter Raffle Helper

This is a simple twitter raffle helper focused on allowing artists that use twitter for raffles to be able to quickly collect the data they need without the need for 3rd party providers or for paying for api's that will used infrequently to facilitate promotional raffles.

## Quickstart

Follow [this](http://tbd) tutorial to collect the data on the tweet. Add them to the root of this project directory and run:

```bash
# Note, for linux and mac users you may have to replace pip/python with pip3/python3
pip install jupyter-lab
python -m jupyter nbconvert --execute --to html results.html
```

You should have a list of eligible twitter accounts in the `eligible_accounts.csv` and a `results.html` file. You can use the reults file directly to make your pick or you can use the `eligible_accounts.csv` for other 3rd party random pickers.

## Data injest 

Next we'll read in the data:

In [3]:
import json

def open_har(file_path):
    with open(file_path, 'r', encoding='utf8') as f:
        return json.load(f)

initial_load = open_har('initial_load.har')
liked_by = open_har('liked_by.har')
retweeted_by = open_har('retweeted_by.har')

## Likes

In [4]:
def try_parse(text):
    try:
        return json.loads(text)
    except:
        return {}

def get_screen_names(raw_data, data_key):
    response_content = [c['response']['content'] for c in raw_data['log']['entries'] if 'https://twitter.com/i/api/graphql' in c['request']['url']]
    parsed_data = [try_parse(c.get('text','{}')) for c in response_content]
    parsed_data = [d for d in parsed_data if 'data' in d]
    screen_names = []
    for data in parsed_data:
        instructions = data.get('data',{}).get(data_key,{}).get('timeline',{}).get('instructions', [])
        for instruction in instructions:
            if instruction['type'] == 'TimelineAddEntries':
                for entry in instruction.get('entries',[]):
                    screen_name = entry.get('content').get('itemContent',{}).get('user_results',{}).get('result',{}).get('legacy',{}).get('screen_name', None)
                    if screen_name:
                        yield screen_name

In [5]:
likes = list(get_screen_names(liked_by, 'favoriters_timeline'))
len(likes)

1054

## Retweets

In [6]:


retweets = list(get_screen_names(retweeted_by, 'retweeters_timeline'))
len(retweets)

167

## Eligible Screen Names

In [8]:
eligible = list(set(retweets).intersection(likes))
len(eligible)

157

## Random Winners

In [12]:
import random

results = []
# NOTE: Need to keep results as a list since set() will force alphabetical order
while len(set(results)) < 1:
    results.append(random.choice(eligible))
display(results)

['CoyoteCocktail']